## RQ: classification accuracy using high level features

In [50]:
# generic imports
import sys
import os
sys.path.append('../')
from definitions import *

#import helper
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
# data structure
import itertools
from collections import Counter,defaultdict

#stats amd ml
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm #https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.LogitResults.html
from scipy import stats
from sklearn import feature_selection
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.metrics import roc_curve

In [53]:
survey_path

'/nfs/juhu/data/rakhasan/bystander-detection/pilot-study2/'

In [57]:
set(feature_df.label)

{0, 1}

In [54]:

human_parts,object_classes =pickle.load(open(survey_path+'human-object-classes.pkl','rb'))
survey_photo_annotations =pickle.load(open(survey_path+'survey_photo_annotations.pkl','rb'))

#annotated body parts 
labeled_body_features = ['_'.join(human_parts[body_part].split()) 
                         for body_part in human_parts.keys()]


photo_df = pickle.load(open(os.path.join(survey_path, 'photo_df.pkl'), 'rb'))
mapping = pickle.load(open(survey_path +'mappings_pilot2','rb'))
feature_df = pickle.load(open(os.path.join(survey_path, 'high-feature-df.pkl'), 'rb'))
feature_df.shape, len(set(photo_df.index.values))

((3802, 10), 4080)

In [55]:
def test_logit(data, predictors, label='label', normalize=True):
    X = data[predictors]
    if normalize:
        X = X.apply(stats.zscore)
    X = sm.add_constant(X)
    y = data[label]
    return sm.Logit(endog=y, exog=X).fit(disp = False)

def print_chisq(model):
    print('\nChisq:{:.2f}, p:{:.2f}\n'.format(model.llr, model.llr_pvalue))
    
def get_Rsq(model):
    return (model.llr) / (- 2*model.llnull)

def get_model_summary(model):
    summary = dict()
    summary['Chi^2'] = model.llr
    summary['p(Chi^2)'] = model.llr_pvalue
    summary['R^2'] = (model.llr) / (- 2*model.llnull)
    return summary

def get_OR(model):
    output = model.conf_int()
    output['OR'] = model.params
    output.columns = ['2.5%', '97.5%', 'OR']
    return np.exp(output)[['OR', '2.5%', '97.5%']]

In [59]:
binary_data = feature_df#[(feature_df.label==1) | (feature_df.label==-1)]
#binary_data['label'] = binary_data.apply(lambda row: 1 if row.label==1 else 0, axis=1)
#binary_data.shape

In [9]:
model = helper.test_logit(binary_data, [pred])
print(model.summary())
helper.get_model_summary(model)
print(pred)

NameError: name 'helper' is not defined

In [35]:
predictors=['posing_num','replacable_num','person_size','photographer_intention_num']
dicts = []
for pred in predictors:
    model = test_logit(binary_data, [pred])
    #print(model.summary())
    d = get_model_summary(model)
    d['Predictor'] = pred
    ors = get_OR(model)
    d['OR'] = ors.loc[pred].OR
    d['2.5%'] = ors.loc[pred]['2.5%']
    d['97.5%'] = ors.loc[pred]['97.5%']
    dicts.append(d)
    
df = pd.DataFrame(dicts)[['Predictor', 'OR', '2.5%', '97.5%','Chi^2', 'p(Chi^2)', 'R^2']].set_index("Predictor").round(2)
df = df.T.rename(columns=high_level_concepts_name).T.sort_values(by='R^2', ascending=False)
df

,OR,2.5%,97.5%,Chi^2,p(Chi^2),R^2
Predictor,,,,,,
Replaceable,0.08,0.06,0.09,2064.40,0.0,0.46
Pose,4.70,4.16,5.32,1007.33,0.0,0.23
Photographer intention,0.53,0.49,0.57,291.40,0.0,0.07
Size,0.64,0.59,0.69,149.10,0.0,0.03


In [11]:
'''use features from study one by one'''
dicts = []
for pred in high_level_concepts_num+img_level_concepts:
    model = helper.test_logit(binary_data, [pred])
    #print(model.summary())
    d = helper.get_model_summary(model)
    d['Predictor'] = pred
    ors = helper.get_OR(model)
    d['OR'] = ors.loc[pred].OR
    d['2.5%'] = ors.loc[pred]['2.5%']
    d['97.5%'] = ors.loc[pred]['97.5%']
    dicts.append(d)
    
df = pd.DataFrame(dicts)[['Predictor', 'OR', '2.5%', '97.5%','Chi^2', 'p(Chi^2)', 'R^2']].set_index("Predictor").round(2)
df = df.T.rename(columns=high_level_concepts_name).T.sort_values(by='R^2', ascending=False)
df

,OR,2.5%,97.5%,Chi^2,p(Chi^2),R^2
Predictor,,,,,,
Replaceable,0.09,0.07,0.10,2254.41,0.0,0.44
Awareness,5.19,4.66,5.78,1476.37,0.0,0.29
Willingness,4.38,3.96,4.86,1247.30,0.0,0.24
Pose,4.48,4.01,5.00,1146.42,0.0,0.22
Comfort,4.05,3.66,4.48,1121.78,0.0,0.22
Size,5.23,4.52,6.05,960.15,0.0,0.19
Distance,0.31,0.29,0.34,930.95,0.0,0.18
Number of people,0.50,0.46,0.54,410.43,0.0,0.08
Photographer intention,0.53,0.49,0.57,330.39,0.0,0.06


In [16]:
df['p(Chi^2)']='$<.0001$'
print(df.to_latex())

\begin{tabular}{lrrrrlr}
\toprule
{} &    OR &  2.5\% &  97.5\% &    Chi\textasciicircum2 &  p(Chi\textasciicircum2) &   R\textasciicircum2 \\
Predictor              &       &       &        &          &           &       \\
\midrule
Replaceable            &  0.09 &  0.07 &   0.10 &  2254.41 &  \$<.0001\$ &  0.44 \\
Awareness              &  5.19 &  4.66 &   5.78 &  1476.37 &  \$<.0001\$ &  0.29 \\
Willingness            &  4.38 &  3.96 &   4.86 &  1247.30 &  \$<.0001\$ &  0.24 \\
Pose                   &  4.48 &  4.01 &   5.00 &  1146.42 &  \$<.0001\$ &  0.22 \\
Comfort                &  4.05 &  3.66 &   4.48 &  1121.78 &  \$<.0001\$ &  0.22 \\
Size                   &  5.23 &  4.52 &   6.05 &   960.15 &  \$<.0001\$ &  0.19 \\
Distance               &  0.31 &  0.29 &   0.34 &   930.95 &  \$<.0001\$ &  0.18 \\
Number of people       &  0.50 &  0.46 &   0.54 &   410.43 &  \$<.0001\$ &  0.08 \\
Photographer intention &  0.53 &  0.49 &   0.57 &   330.39 &  \$<.0001\$ &  0.06 \\
Photo plac

In [19]:
'''Classify using the variables found from factor analysis. Try one predictor at a time from the two groups'''
for v1 in ['was_aware_num', 'posing_num', 'comfort_num', 'will_num']:
    for v2 in img_level_concepts:
        model = helper.test_logit(binary_data, predictors=[v1,'replacable_num',v2])
        print(model.summary())
        print(helper.get_OR(model))
        print('\nR^2:',helper.get_model_summary(model)['R^2'])
        print('\n\n')

                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 3802
Model:                          Logit   Df Residuals:                     3798
Method:                           MLE   Df Model:                            3
Date:                Wed, 10 Apr 2019   Pseudo R-squ.:                  0.5242
Time:                        16:08:58   Log-Likelihood:                -1216.3
converged:                       True   LL-Null:                       -2556.4
                                        LLR p-value:                     0.000
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.0784      0.061     17.653      0.000       0.959       1.198
was_aware_num      0.9809      0.069     14.251      0.000       0.846       1.116
replacable_num    -2.0071      0.082

In [69]:
'''Using the most intuitive predictors: posing, size, and replaceable '''
model = test_logit(binary_data, predictors=['was_aware_num','replacable_num',
                                                   'person_size'])
print(model.summary())
or_model = get_OR(model)
print(or_model)
print('\nR^2:',get_model_summary(model)['R^2'])

                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 3802
Model:                          Logit   Df Residuals:                     3798
Method:                           MLE   Df Model:                            3
Date:                Thu, 13 Feb 2020   Pseudo R-squ.:                  0.5148
Time:                        15:55:04   Log-Likelihood:                -1240.4
converged:                       True   LL-Null:                       -2556.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.9930      0.058     17.192      0.000       0.880       1.106
was_aware_num      1.1198      0.066     16.884      0.000       0.990       1.250
replacable_num    -2.0844      0.081

In [67]:
'''Using the most intuitive predictors: posing, size, and replaceable '''
model = test_logit(binary_data, predictors=['posing_num','replacable_num',
                                                   'person_size','photographer_intention_num'])
print(model.summary())
or_model = get_OR(model)
print(or_model)
print('\nR^2:',get_model_summary(model)['R^2'])
print()
print(or_model.T.rename(columns=high_level_concepts_name).T.round(2).to_latex())

                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 3802
Model:                          Logit   Df Residuals:                     3797
Method:                           MLE   Df Model:                            4
Date:                Thu, 13 Feb 2020   Pseudo R-squ.:                  0.5216
Time:                        13:37:33   Log-Likelihood:                -1222.9
converged:                       True   LL-Null:                       -2556.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          1.0681      0.060     17.837      0.000       0.951       1.185
posing_num                     1.0692      0.070     15.368      0.000       0.933 

In [19]:
'''Using the most intuitive predictors: posing, size, and replaceable '''
model = helper.test_logit(binary_data, predictors=['posing_num','replacable_num',
                                                   'person_size','photographer_intention_num'])
print(model.summary())
or_model = helper.get_OR(model)
print(or_model)
print('\nR^2:',helper.get_model_summary(model)['R^2'])
print()
print(or_model.T.rename(columns=high_level_concepts_name).T.round(2).to_latex())

                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 3802
Model:                          Logit   Df Residuals:                     3797
Method:                           MLE   Df Model:                            4
Date:                Thu, 24 Oct 2019   Pseudo R-squ.:                  0.5331
Time:                        02:03:31   Log-Likelihood:                -1193.7
converged:                       True   LL-Null:                       -2556.4
                                        LLR p-value:                     0.000
                                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
const                          1.1589      0.063     18.293      0.000         1.035     1.283
posing_num                     0.9165      0.072     12.717      0.000         0.77

In [7]:
model = helper.test_logit(binary_data, predictors=high_level_concepts_num)
print(model.summary())
print(helper.get_OR(model))
print('\nR^2:',helper.get_model_summary(model)['R^2'])

                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 3802
Model:                          Logit   Df Residuals:                     3794
Method:                           MLE   Df Model:                            7
Date:                Tue, 09 Apr 2019   Pseudo R-squ.:                  0.5415
Time:                        22:52:24   Log-Likelihood:                -1172.2
converged:                       True   LL-Null:                       -2556.4
                                        LLR p-value:                     0.000
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          1.1741      0.065     18.185      0.000       1.048       1.301
was_aware_num                  0.5724      0.124      4.609      0.000       0.329 

In [22]:
import cv2 

cropped_img = dict()
IMG_SIZE = (256, 256)
image_names = list(set(feature_df.index.values))
for idx in image_names:
    cropped_img[idx] = cv2.resize(cv2.imread(survey_path+'cropped-photos/'+idx+'.jpg'), 
                   dsize=IMG_SIZE, interpolation=cv2.INTER_CUBIC)

In [24]:
feature_df['resized_cropped_img']

,was_aware_num,posing_num,comfort_num,will_num,photographer_intention_num,replacable_num,photo_place_num,subject_bystander_num,label,person_distance_axes_norm,person_distance_img_norm,person_size,num_people
photo_no,,,,,,,,,,,,,
1,2.500000,2.000000,2.000000,1.500000,2.50,-2.50,2.000000,1.75,1,0.116905,0.000242,0.622600,1
10,-1.333333,-1.333333,1.666667,1.333333,-2.00,3.00,-1.666667,-1.00,-1,0.326488,0.295294,0.002456,4
100,0.750000,0.000000,0.500000,0.500000,1.25,-0.25,1.250000,1.50,1,0.261331,0.002807,0.104219,1
1000,2.000000,1.750000,2.500000,2.000000,-2.25,-2.25,-1.000000,1.50,1,0.029659,0.000154,0.275583,3
1001,1.500000,0.000000,1.500000,1.000000,-0.25,1.25,-2.000000,-0.75,-1,0.327691,0.018888,0.021361,2


In [25]:
c=pd.Series(cropped_img)

In [27]:
c.head()

1       [[[6, 255, 7], [19, 239, 21], [26, 243, 23], [...
10      [[[0, 255, 2], [0, 255, 2], [0, 255, 2], [0, 2...
100     [[[5, 255, 0], [6, 255, 0], [6, 255, 0], [5, 2...
1000    [[[6, 254, 0], [2, 255, 4], [7, 248, 6], [5, 2...
1001    [[[4, 255, 0], [4, 255, 0], [4, 255, 0], [4, 2...
dtype: object